In [1]:
import cantera as ct
import os
import shutil
import pandas as pd
import numpy as np

In [3]:
or_p = '/work/westgroup/chao/bm_project/base_original/binding_energies/12.0_c-5.75o-3.75/cantera.yaml'
bm_p = '/work/westgroup/chao/bm_project/base_bm/binding_energies/12.0_c-5.75o-3.75/cantera.yaml'

gas_or = ct.Solution(or_p, 'gas')
surf_or = ct.Interface(or_p, 'surface1', [gas_or])
gas_or.species()


/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in cp/R detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  10.615319744584205
	Value computed using high-temperature polynomial: 10.773680323194302

  """
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -16.268781899829992
	Value computed using high-temperature polynomial: -16.280689980057943

  """
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: NasaPoly2::validate: 
For species CH4OX(43), discontinuity in cp/R detected at Tmid = 850.41
	Value computed using low-temperature polynomial:  11.305535344771027
	Value computed using high-temperature polyn

[<Species Ar>,
 <Species Ne>,
 <Species N2>,
 <Species CH4(2)>,
 <Species O2(3)>,
 <Species CO2(4)>,
 <Species H2O(5)>,
 <Species H2(6)>,
 <Species CO(7)>,
 <Species C2H6(8)>,
 <Species CH2O(9)>,
 <Species CH3(10)>,
 <Species C3H8(11)>,
 <Species H(12)>,
 <Species C2H5(13)>,
 <Species CH3OH(14)>,
 <Species HCO(15)>,
 <Species CH3CHO(16)>,
 <Species OH(17)>,
 <Species C2H4(18)>,
 <Species CH3OO(20)>,
 <Species C2H4(62)>]

In [10]:
benchmark_p = '/work/westgroup/chao/bm_project/base_original/binding_energies/12.0_c-5.75o-3.75/gas_profiles/gas_out0.6.csv'
df_benchmark = pd.read_csv(benchmark_p)

ch4_benchmark_1045 = df_benchmark.iloc[3, 1:][1045]
diffs = []
for i in range(surf_or.n_reactions):
    p_target = f'bm2or/rxn{i}/gas_profiles/gas_out0.6.csv'
    df_target = pd.read_csv(p_target)
    ch4_bm_1045 = df_target.iloc[3, 1:][1045]
    diff = abs((ch4_bm_1045 - ch4_benchmark_1045) / ch4_benchmark_1045)
    diffs.append(diff)
    if diff > 0.5:
        print(i)
        print(surf_or.reaction(i))

top_10_diffs = np.argsort(diffs)[-10:]
for i in top_10_diffs:
    print(diffs[i])
    print(i)
    print(surf_or.reaction(i))

0.0009262019689140601
25
H2OX(32) + OX(25) <=> 2 OHX(31)
0.0022083175427806434
19
CH3X(27) + OX(25) <=> CH2X(26) + OHX(31)
0.0024759869307781415
38
CH2O(9) + 2 X(1) <=> CH2X(26) + OX(25)
0.0046980032063791135
68
CH3OH(14) + 2 X(1) <=> CH3O_X(45) + HX(21)
0.0050308203819536945
70
CH4X(24) + OX(25) <=> CH3O_X(45) + HX(21)
0.015587392881951904
15
CH4(2) + OX(25) + X(1) <=> CH3X(27) + OHX(31)
0.032423401391385796
36
C2H6(8) + 2 X(1) <=> 2 CH3X(27)
0.03507281678514591
32
CH4(2) + X(1) <=> CH4X(24)
0.1212739309954735
39
CH3OH(14) + 2 X(1) <=> CH3X(27) + OHX(31)
0.1894458896523358
35
H2O(5) + 2 X(1) <=> HX(21) + OHX(31)


In [11]:
# check the rate of progress
or_rp = '/work/westgroup/chao/bm_project/base_original/binding_energies/12.0_c-5.75o-3.75/rates_of_progress/net_rates_0.6.csv'
bm_rp = '/work/westgroup/chao/bm_project/base_original/binding_energies/12.0_c-5.75o-3.75/bm2or/rxn39/rates_of_progress/net_rates_0.6.csv'
df_or_rp = pd.read_csv(or_rp)
df_bm_rp = pd.read_csv(bm_rp)

print(df_or_rp.iloc[1045,36], df_bm_rp.iloc[1045,36])

-0.0013822061768341712 -0.00042095755595143
